In [115]:
from importlib import reload
import sys
import util
reload(util)
data = util.FontData
FontDataset = util.FontDataset
import experiment
reload(experiment)
from sklearn import linear_model, neural_network

In [116]:
data.load()

Loading embeddings...done
Loading typographic + semantic vectors...done
Loading images...done
Loading glyphs...done
Loading SVGs...done


In [117]:
train_dataset = FontDataset(data, 'train')
val_dataset = FontDataset(data, 'val')

In [118]:
train_typographic = data.get_all_typographic('train')
val_typographic = data.get_all_typographic('val')

In [119]:
train_image = data.get_all_image('train')
train_image = train_image.reshape(train_image.shape[0], -1)
val_image = data.get_all_image('val')
val_image = val_image.reshape(val_image.shape[0], -1)

In [120]:
train_semantic = data.get_all_semantic('train')
val_semantic = data.get_all_semantic('val')

### Linear Regression: Image -> Typographic

In [121]:
image_typographic_reg = linear_model.LinearRegression()

In [122]:
image_typographic_reg.fit(train_image, train_typographic)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [123]:
train_stats, train_typographic_pred = experiment.run_experiment(image_typographic_reg, train_image, train_typographic)

{'RMSE': 1.3667070561496075e-05, 'R2': 0.9999999998025613}


In [124]:
val_stats, val_typographic_pred = experiment.run_experiment(image_typographic_reg, val_image, val_typographic)

{'RMSE': 1.3457238674163818, 'R2': -0.6625254045401793}


### Linear Regression: Typographic -> Semantic

In [125]:
typographic_semantic_reg = linear_model.LinearRegression()

In [126]:
typographic_semantic_reg.fit(train_typographic, train_semantic)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [127]:
train_stats, train_semantic_pred = experiment.run_experiment(typographic_semantic_reg, train_typographic, train_semantic)

{'RMSE': 0.6425207853317261, 'R2': 0.5791966258956402}


In [128]:
val_stats, val_semantic_pred = experiment.run_experiment(typographic_semantic_reg, val_typographic, val_semantic)

{'RMSE': 0.652705729007721, 'R2': 0.5643953306184701}


### Linear Regressions: Image -> Typographic -> Semantic

In [129]:
class SequentialLinearRegression:
    def __init__(self, reg1, reg2):
        self.reg1 = reg1
        self.reg2 = reg2
    def predict(self, X):
        return self.reg2.predict(self.reg1.predict(X))

image_typographic_semantic_reg = SequentialLinearRegression(image_typographic_reg, typographic_semantic_reg)

In [130]:
train_stats, train_semantic_pred = experiment.run_experiment(image_typographic_semantic_reg, train_image, train_semantic)

{'RMSE': 0.6425206065177917, 'R2': 0.5791968681664224}


In [131]:
val_stats, val_semantic_pred = experiment.run_experiment(image_typographic_semantic_reg, val_image, val_semantic)

{'RMSE': 1.2455190420150757, 'R2': -0.617107781671052}


### Linear Regression: Image -> Semantic

In [132]:
image_semantic_reg = linear_model.LinearRegression()

In [133]:
image_semantic_reg.fit(train_image, train_semantic)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [134]:
train_stats, train_semantic_pred = experiment.run_experiment(image_semantic_reg, train_image, train_semantic)

{'RMSE': 1.5096046809048858e-05, 'R2': 0.9999999997676309}


In [135]:
val_stats, val_semantic_pred = experiment.run_experiment(image_semantic_reg, val_image, val_semantic)

{'RMSE': 1.4704996347427368, 'R2': -1.250469831432338}


### Multilayer Perceptron: Image -> Semantic

In [136]:
image_semantic_nn = neural_network.MLPRegressor(hidden_layer_sizes=(10,))

In [ ]:
image_semantic_nn.fit(train_image, train_semantic)

In [ ]:
train_stats, train_semantic_pred = experiment.run_experiment(image_semantic_nn, train_image, train_semantic)

In [ ]:
val_stats, val_semantic_pred = experiment.run_experiment(image_semantic_nn, val_image, val_semantic)

#### Example

In [ ]:
val_semantic_pred[0]

In [ ]:
data.get_semantic(data.get_name(0, "val"))